In [1]:
from packages.serial_link_robot import *
import sympy as sp

In [2]:
theta1, theta2,theta3, D3, d1, k1, k2, k3, d3 = sp.symbols('theta1, theta2,theta3, D3, d1, k1, k2, k3, d3')
robot = SerialLinkRobot()
robot.add_revolute_joint(theta1, d1, 0, sp.pi/2)
robot.add_revolute_joint(theta2, 0, k1, sp.pi/2)
robot.add_prismatic_joint(0, D3, 2, 0)
robot.add_revolute_joint(theta3, 0, 2, 0)
robot.add_subs([(d1, 2), (k1, 2)])
robot.interact()

error: Not connected to physics server.